<a href="https://colab.research.google.com/github/KBlack-HIAMS/CarChecker/blob/master/Data_Analytics_OP130_Alum_Contamination_r2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imports for data file selection and processing
from ipywidgets import *
from ipywidgets import Button
from IPython.display import clear_output, display
import tkinter as tk
from tkinter import Tk, filedialog, messagebox
import os
from datetime import datetime
import re

# General imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

class DataFilename:
    def __init__(self,filename):
        self.filename = filename
        

def select_files(b):
    filetypes = (
        ('comma-delimited','*.csv'),
        ('text files','*.txt')
    )
    clear_output()                                         # Button is deleted after it is clicked.
    root = Tk()
    root.withdraw()                                        # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True)   # Raise the root to the top of all windows.
    b.files = filedialog.askopenfilename(
        title='Please select target data files...',
        filetypes=filetypes,multiple=True
    )                                                      # List of selected files will be set button's file attribute.
    print(b.files)                                         # Print the list of files selected.
    fName.filename = b.files                               # Assign user selected filename to class instance variable
    
def select_file():
    filetypes = (
        ('comma-delimited','*.csv'),
        ('text files','*.txt')
    )
    
    fName.filename = filedialog.askopenfilename(
        title='Please select target data file...',
        filetypes=filetypes,
        multiple=False
    )

def msgbox_showMsg(msgTitle, msgText):
    clear_output()                                         # Button is deleted after it is clicked.
    root = Tk()
    root.withdraw()                                        # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True)   # Raise the root to the top of all windows.
    tk.messagebox.showinfo(title=msgTitle,message=msgText)
    
def read_dataFile(df_out):
    # selected file path list has been saved into class variable fName.filename
    # iterate over file path list, opening each file, and appending file data to master data file
    # we want to combine all files into one data file, while keeping the header information (without duplication)
    
    if(len(fName.filename)>0):
        # initialize master dataframe for appending using first file read
        df_out = pd.read_csv(fName.filename[0])
        
        if(len(fName.filename)>1):
            for i in range(1,len(fName.filename)):
                df_inFile = pd.read_csv(fName.filename[i])
                df_out = pd.concat([df_out,df_inFile], ignore_index=True)
                
    return df_out



# **==============================================================================**
# start of script
# create file select instance
fName = DataFilename('')

# create empty dataframe for reading in data from previously processed datafile
df_op130_cleaned = pd.DataFrame()


# prompt user to select target data file
msgbox_showMsg("Info","Please select target OP130 data file(s)...")

fileselect = Button(description="File select")
fileselect.on_click(select_files)
display(fileselect)

In [ ]:
# selected file path list has been saved into class variable fName.filename

df_op130_cleaned = read_dataFile(df_op130_cleaned)

In [ ]:
df_op130_cleaned.head()

# Select Analysis Output Type
We need to view/plot VDP4 Assembly defect data with respect to pump assembly date and alternatively to BODY-FRONT machining date.

## Sort Analysis w.r.t. Pump Assembly Date

In [ ]:
pivot_count = df_op130_cleaned.pivot_table(values='workid', 
                                           index='Assy_week', 
                                           columns='OP130_Result', 
                                           aggfunc=pd.Series.nunique, 
                                           fill_value='0')

In [ ]:
# verify output
print(pivot_count)

In [ ]:
# save pivot table results to excel (.csv) file
now = datetime.now()
f_time = now.strftime('%Y%m%d-%H%M%S')
f_name = 'OP130A_pivot-data_r2-' + f_time + '.csv'
pivot_count.to_csv(f_name,index=True)

## Sort Analysis w.r.t. BODY-FRONT Machining Date

In [ ]:
pivot_count = df_op130_cleaned.pivot_table(values='workid', 
                                           index='BODY-FRONT_Mach.Date', 
                                           columns='OP130_Result', 
                                           aggfunc=pd.Series.nunique, 
                                           fill_value='0')

In [ ]:
# save pivot table results to excel (.csv) file
now = datetime.now()
f_time = now.strftime('%Y%m%d-%H%M%S')
f_name = 'OP130M_pivot-data_r2-' + f_time + '.csv'
pivot_count.to_csv(f_name,index=True)

In [ ]:
# verify output
print(pivot_count)